In [22]:
import json
from pytrends.request import TrendReq
import pandas as pd
import numpy as np
import time
startTime = time.time()
pytrend = TrendReq(hl='es-SV', tz=360)

In [23]:
class MyTrendReq(TrendReq):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def interest_by_region(self, resolution='COUNTRY', inc_low_vol=False,
                           inc_geo_code=False):
        """Request data from Google's Interest by Region section and return a dataframe"""

        # Make the request
        region_payload = dict()

        if self.geo == '': 
            self.interest_by_region_widget['request']['resolution'] = resolution 
        elif self.geo == 'SV' and resolution in ['CITY', 'REGION']: 
            self.interest_by_region_widget['request']['resolution'] = resolution        

        self.interest_by_region_widget['request'][
            'includeLowSearchVolumeGeos'] = inc_low_vol

        # Convert to string as requests will mangle
        region_payload['req'] = json.dumps(
            self.interest_by_region_widget['request'])
        region_payload['token'] = self.interest_by_region_widget['token']
        region_payload['tz'] = self.tz

        # Parse returned json
        req_json = self._get_data(
            url=TrendReq.INTEREST_BY_REGION_URL,
            method=TrendReq.GET_METHOD,
            trim_chars=5,
            params=region_payload,
        )
        df = pd.DataFrame(req_json['default']['geoMapData'])
        if (df.empty):
            return df

        # Rename the column with the search keyword
        df = df[['geoName', 'coordinates', 'value']].set_index(['geoName']).sort_index()
      
        # Split list columns into seperate ones
        result_df = df.copy()
        result_df = df['value'].apply(lambda x: pd.Series(
            str(x).replace('[', '').replace(']', '').split(',')))
        result_df1 = df['coordinates'].apply(lambda x: pd.Series(
            str(x).replace('[', '').replace(']', '').replace("{'lat':", '').replace('}', '').replace("'lng':", '').split(',')))
        if inc_geo_code:
            result_df['geoCode'] = df['geoCode']
            
        # Rename each column with its search term    
        for idx, kw in enumerate(self.kw_list):
            result_df[kw] = result_df[idx].astype('int')
            del result_df[idx]
        result_final = pd.merge(result_df, result_df1, on="geoName")
        
        # Reset index and rename columns
        result_final1 = result_final.copy()
        result_final1.reset_index(inplace=True)
        result_final1.rename(columns={"geoName": "Ciudad", 0:"Latitud", 1: "Longitud"}, inplace=True)
        
        
        for i in ['Latitud','Longitud']:
            result_final1[i] = result_final1[i].astype(np.float64)
            
        return result_final1

In [24]:
def excel_writer(data_frame, *, sheet_name='Sheet1',table_name='IOT'):
    data_frame.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=False)
    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Get the dimensions of the dataframe.
    (max_row, max_col) = data_frame.shape

    # Create a list of column headers, to use in add_table().
    column_settings = [{'header': column} for column in data_frame.columns]

    # Add the Excel table structure. Pandas will add the data.
    worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings,'name': table_name})

    # Make the columns wider for clarity.
    worksheet.set_column(0, max_col - 1, 12)

In [25]:
# Add a list of keywords for searching on Trends and define the timeframe
keywords = ['asamblea']
timeframe = '2020-05-14 2021-05-14'
dataset = []
related = []
for x in range(0,len(keywords)):
     keywords1 = [keywords[x]]
     pytrend.build_payload(
     kw_list=keywords1,
     cat=0,
     timeframe=timeframe,   
     geo='SV')
     data = pytrend.interest_over_time()
     related.append(pytrend.related_queries() )  
     if not data.empty:
          data = data.drop(labels=['isPartial'],axis='columns')
          dataset.append(data)

In [27]:
result = pd.concat(dataset, axis=1)
marca_names = list(result.columns.values)
df_marca = pd.DataFrame(marca_names,columns =['marca'])

In [28]:
# Change the date format and reset the index
result.index = result.index.strftime('%d/%m/%Y')
result.reset_index(inplace=True)

In [29]:
# Unpivot the result dataframe, sort and rename the column date
df_unpivoted = result.melt(id_vars=['date'], var_name='marca', value_name='valor')
df_unpivoted.sort_values(by=['date'], inplace = True)
df_unpivoted.rename(columns={"date": "Semana"}, inplace=True)

In [30]:
 # Get the name of all the search keywords and create a dataframe based on top and rising terms 
indexes =[index for index, value in enumerate(marca_names)]
dict1 = dict(zip(marca_names, indexes))
listDataframes = []
for k,v in dict1.items():
    df_marca_name = pd.DataFrame({'Name': [k for i in range(len(related[v][k]['top']))]})
    df_marca_rising = related[v][k]['rising']
    df_marca_rising.drop(columns=['value'], inplace=True)
    df_marca_rising = df_marca_rising.assign(N=[x for x in range(1,len(df_marca_rising)+1)])
    df_marca_merge = pd.concat([df_marca_name,df_marca_rising],ignore_index=True, axis=1)
    df_marca_top = related[v][k]['top']
    df_merge_total = pd.merge(df_marca_merge,df_marca_top,left_index=True, right_index=True)
    df_merge_total = df_merge_total.rename(columns={"query": "TOP", 0:"marca", 1:"RISING", 2: "N", "value":"%"})
    listDataframes.append(df_merge_total)
df_related_queries = pd.concat(listDataframes).reset_index(drop=True) 

In [31]:
 # Generate a new object based on MyTrendReq class
pytrend_geo = MyTrendReq()
pytrend_geo.build_payload(kw_list=keywords, geo='SV', timeframe=timeframe)  
df = pytrend_geo.interest_by_region(resolution='CITY', inc_low_vol=True, inc_geo_code=False)
df_unpivoted_geo = df.melt(id_vars=['Ciudad','Latitud','Longitud'], var_name='marca', value_name='valor')

In [32]:
 # Invoke the excelWriter engine
writer = pd.ExcelWriter('pandas_table.xlsx', engine='xlsxwriter',date_format='dd mm yyyy')

In [33]:
 # Call the excel_writer function
excel_writer(df_unpivoted)
excel_writer(df_marca,sheet_name='Sheet2',table_name='marca')
excel_writer(df_related_queries,sheet_name= 'TOP_RISING',table_name='TOP_RISING')
excel_writer(df_unpivoted_geo,sheet_name= 'GEO',table_name='GEO')

In [34]:
 # Close the Pandas Excel writer and output the Excel file.
writer.save()